In [39]:
# %pip install xgboost
# %pip install scikit-learn

In [2]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import pandas as pd
import csv
import os
import numpy as np
import xgboost as xgb
from sklearn.utils import resample
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, confusion_matrix
import pickle
from sklearn.model_selection import GridSearchCV

In [3]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
dataset_train = pd.read_csv('./train.csv')
dataset_test = pd.read_csv('./test.csv')
#preprocessing

#correcting video directory
dataset_train['clip_path'] = dataset_train['clip_path'].apply(lambda x: '.' + x)
dataset_test['clip_path'] = dataset_test['clip_path'].apply(lambda x: '.' + x)



#Downsampling to make the dataset balanced and normally distributed

def downsample(df, min_samples_per_class):
    downsampled_dfs = []

    # Get the unique classes
    classes = df['label'].unique()

    for cls in classes:
        class_df = df[df['label'] == cls]

        # Ensure the number of samples doesn't exceed available data
        n_samples = min(min_samples_per_class, len(class_df))

        # Downsample the class to the specified number of samples
        downsampled_class_df = resample(
            class_df,
            replace=False,  # sample without replacement
            n_samples=n_samples,  # to match the minimum class size
            random_state=42  # for reproducibility
        )
        downsampled_dfs.append(downsampled_class_df)

    return pd.concat(downsampled_dfs)

dataset_train = downsample(dataset_train, 40)
dataset_test = downsample(dataset_test,40)



In [4]:
print(dataset_train['label'].value_counts())
print(dataset_test['label'].value_counts())

label
Swing                40
PushUps              40
Hammering            40
TrampolineJumping    40
PizzaTossing         40
                     ..
PlayingDhol          40
Archery              40
Punch                40
VolleyballSpiking    40
Diving               40
Name: count, Length: 101, dtype: int64
label
Basketball           34
TennisSwing          21
CricketShot          21
BoxingPunchingBag    21
PlayingDhol          21
                     ..
Skijet               13
UnevenBars           13
PushUps              13
TaiChi               13
PlayingViolin        13
Name: count, Length: 101, dtype: int64


In [43]:
def process_videos(dataset, set_name, amount):
    # Initialize the Holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:

        landmarks = ['class']
        for val in range(1, 33+1):
            landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
        with open(f'{set_name}_coords.csv', mode='w', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(landmarks)

        for i in range(len(dataset[:amount])):
            video_path = dataset['clip_path'].iloc[i]
            class_name = dataset['label'].iloc[i]
            print(f"Processing {set_name} video file: {video_path}")

            cap = cv2.VideoCapture(video_path)

            if not cap.isOpened():
                print(f"Error: Could not open video file {video_path}")
                continue  # Skip to the next video file if there is an error

            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                frame_count += 1

                if not ret or frame is None:
                    print(f"Error: Could not read frame {frame_count} from {video_path}")
                    break

                try:
                    # Resize the frame to a fixed size for consistency
                    frame = cv2.resize(frame, (640, 480))

                    # Check the frame's data type and convert to uint8 if necessary
                    if frame.dtype != 'uint8':
                        frame = (frame * 255).astype('uint8')

                    # Apply Median Blur to reduce noise
                    frame = cv2.medianBlur(frame, 5)

                    # Convert to grayscale for histogram equalization
                    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                    # Apply Histogram Equalization
                    equalized_frame = cv2.equalizeHist(gray_frame)

                    # Convert back to BGR
                    frame = cv2.cvtColor(equalized_frame, cv2.COLOR_GRAY2BGR)

                    # Recolor Feed (convert BGR to RGB)
                    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                except cv2.error as e:
                    print(f"Error during preprocessing at frame {frame_count}: {e}")
                    break

                image.flags.writeable = False

                # Make Detections
                results = holistic.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Draw pose landmarks
                if results.pose_landmarks:  # Check if pose landmarks are present
                    mp_drawing.draw_landmarks(
                        image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                    )

                    # Export coordinates
                    try:
                        # Extract Pose landmarks
                        pose = results.pose_landmarks.landmark
                        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

                        # Append class name
                        pose_row.insert(0, class_name)

                        # Export to CSV
                        with open(f'{set_name}_coords.csv', mode='a', newline='') as f:
                            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                            csv_writer.writerow(pose_row)

                    except Exception as e:
                        print(f"Error extracting or writing pose landmarks at frame {frame_count}: {e}")

            cap.release()

# Process training videos
process_videos(dataset_train, 'train', 140)

# Process testing videos
process_videos(dataset_test, 'test', 60)

cv2.destroyAllWindows()


Processing train video file: ./train/Swing/v_Swing_g15_c03.avi
Error: Could not read frame 89 from ./train/Swing/v_Swing_g15_c03.avi
Processing train video file: ./train/Swing/v_Swing_g15_c06.avi
Error: Could not read frame 202 from ./train/Swing/v_Swing_g15_c06.avi
Processing train video file: ./train/Swing/v_Swing_g14_c03.avi
Error: Could not read frame 152 from ./train/Swing/v_Swing_g14_c03.avi
Processing train video file: ./train/Swing/v_Swing_g01_c04.avi
Error: Could not read frame 127 from ./train/Swing/v_Swing_g01_c04.avi
Processing train video file: ./train/Swing/v_Swing_g05_c06.avi
Error: Could not read frame 202 from ./train/Swing/v_Swing_g05_c06.avi
Processing train video file: ./train/Swing/v_Swing_g14_c01.avi
Error: Could not read frame 114 from ./train/Swing/v_Swing_g14_c01.avi
Processing train video file: ./train/Swing/v_Swing_g11_c05.avi
Error: Could not read frame 152 from ./train/Swing/v_Swing_g11_c05.avi
Processing train video file: ./train/Swing/v_Swing_g09_c03.avi


In [5]:
df_train = pd.read_csv('./train_coords.csv')
df_test = pd.read_csv('./test_coords.csv')

In [6]:
x_train = df_train.drop('class', axis=1)
y_train = df_train['class']
x_test = df_test.drop('class', axis=1)
y_test = df_test['class']

In [7]:
#before gridsearch
pipelines = {
    # 'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    # 'rc':make_pipeline(StandardScaler(), RidgeClassifier())
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier())
    # 'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
    # 'knn':make_pipeline(StandardScaler(), KNeighborsClassifier()),
    # 'xgb':make_pipeline(StandardScaler(), xgb.XGBClassifier()),
}

In [8]:
    #after gridsearch
    pipelines = {
        # 'lr':make_pipeline(StandardScaler(), LogisticRegression(C=0.1, penalty='l2', solver='liblinear')),
        # 'rc':make_pipeline(StandardScaler(), RidgeClassifier(alpha=10))
        'rf':make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=30, min_samples_leaf=1, min_samples_split=10, n_estimators=50)),
        # 'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier(learning_rate=0.1, max_depth=5, n_estimators=200)),
    }

In [9]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, f1_score

# Assuming pipelines is a dictionary of algorithms and their pipelines
fit_models = {}
for algo, pipeline in pipelines.items():
    print(f"Training pipeline for: {algo}")

    if x_train.shape[0] == 0 or y_train.shape[0] == 0:
        print(f"Error: No training data for {algo}. Skipping...")
        continue

    try:
        # Perform cross-validation to reduce overfitting
        scores = cross_val_score(pipeline, x_train, y_train, cv=5, scoring='accuracy')
        print(f"Cross-Validation Accuracy for {algo}: {scores.mean():.4f} ± {scores.std():.4f}")

        # Fit the pipeline on the full training data
        model = pipeline.fit(x_train, y_train)
        fit_models[algo] = model
        print(f"Successfully trained {algo}")
    except Exception as e:
        print(f"Error training {algo}: {e}")

# Evaluate models
for algo, model in fit_models.items():
    print(f"Evaluating model for: {algo}")

    # Training metrics
    yhat_train = model.predict(x_train)
    train_accuracy = accuracy_score(y_train, yhat_train)
    train_precision = precision_score(y_train, yhat_train, average='weighted')
    train_f1 = f1_score(y_train, yhat_train, average='weighted')
    print(f"{algo} - Training Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, F1-Score: {train_f1:.4f}")

    # Testing metrics
    yhat = model.predict(x_test)
    test_accuracy = accuracy_score(y_test, yhat)
    test_precision = precision_score(y_test, yhat, average='weighted')
    test_f1 = f1_score(y_test, yhat, average='weighted')
    print(f"{algo} - Test Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, F1-Score: {test_f1:.4f}")

    # Overfitting warning
    if train_accuracy - test_accuracy > 0.1:
        print(f"Warning: {algo} might be overfitting (Train-Test Gap > 10%)")


Training pipeline for: rf
Cross-Validation Accuracy for rf: 0.8002 ± 0.0821
Successfully trained rf
Evaluating model for: rf
rf - Training Accuracy: 0.9991, Precision: 0.9991, F1-Score: 0.9991
rf - Test Accuracy: 0.7210, Precision: 0.7858, F1-Score: 0.7342


In [10]:
with open('limb_detection.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [11]:
with open('limb_detection.pkl', 'rb') as f:
    model = pickle.load(f)

In [54]:
# %pip install gradio

  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/57.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/57.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/57.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/57.2 MB ? eta -:--:--
    --------------------------------------- 0.8/57.2 MB 1.5 MB/s eta 0:00:39
    --------------------------------------- 1.0/57.2 MB 1.4 MB/s eta 0:00:41
    --------------------------------------- 1.3/57.2 MB 1.3 MB/s eta 0:00:44
   - -------------------------------------- 1.8/57.2 MB 1.3 MB/s eta 0:00:42
   - -------------------------------------- 1.8/57.2 MB 1.3 MB/s eta 0:00:42
   - -------------------------------------- 2.1/57.2 MB 1.2 MB/s eta 0:00:45
   - -------------------------------------- 2.1/57.2 MB 1.2 MB/s eta 0:00:45
   - -------------------------------------- 2.4/57.2 MB 1.0 MB/s eta 0:00:54
   - -----------------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import gradio as gr

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def test_videos(dataset, amount):
    # Initialize the Holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
        for i in range(min(len(dataset), amount)):
            video_path = dataset['clip_path'].iloc[i]
            class_name = dataset['label'].iloc[i]

            cap = cv2.VideoCapture(video_path)

            if not cap.isOpened():
                print(f"Error: Could not open video file {video_path}")
                continue  # Skip to the next video file if there is an error

            print(f"Processing video: {video_path} ({class_name})")
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret or frame is None:
                    print(f"End of video or error reading frame {frame_count} from {video_path}")
                    break

                frame_count += 1
                try:
                    # Resize frame
                    frame = cv2.resize(frame, (640, 480))

                    # Convert to uint8 if necessary
                    if frame.dtype != np.uint8:
                        frame = (frame * 255).astype(np.uint8)

                    # Apply Median Blur
                    frame = cv2.medianBlur(frame, 5)

                    # Convert to grayscale and apply Histogram Equalization
                    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    equalized_frame = cv2.equalizeHist(gray_frame)
                    frame = cv2.cvtColor(equalized_frame, cv2.COLOR_GRAY2BGR)

                    # Convert to RGB for MediaPipe
                    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    image.flags.writeable = False

                    # Process the frame with MediaPipe Holistic
                    results = holistic.process(image)
                    image.flags.writeable = True
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                    # Draw pose landmarks if present
                    if results.pose_landmarks:
                        mp_drawing.draw_landmarks(
                            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                        )

                        # Extract pose landmarks
                        try:
                            pose = results.pose_landmarks.landmark
                            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

                            x = pd.DataFrame([pose_row])
                            body_language_class = model.predict(x)[0]
                            body_language_prob = model.predict_proba(x)[0]
                            print(f"Class: {body_language_class}, Probability: {body_language_prob}")

                            # Annotate the frame
                            cv2.putText(image, f'Class: {body_language_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                            cv2.putText(image, f'Probability: {body_language_prob[np.argmax(body_language_prob)]:.2f}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                        except Exception as e:
                            print(f"Error extracting pose landmarks: {e}")

                    # Display the video frame
                    cv2.imshow('Processed Video Feed', image)

                    # Break the loop on 'q' key press
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                except cv2.error as e:
                    print(f"OpenCV error at frame {frame_count}: {e}")
                    break

            cap.release()
            print(f"Finished processing video: {video_path}")

    cv2.destroyAllWindows()

def gradio_interface():
    return test_videos(dataset_test, 20)

iface = gr.Interface(
    fn=gradio_interface,
    inputs=None,
    outputs=gr.Video(label="Labelled Video"),
    live=True,
    description="Click the button to process the video."
)

iface.launch()

In [13]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import gradio as gr

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def test_videos(dataset, amount):
    # Initialize the Holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
        for i in range(min(len(dataset), amount)):
            video_path = dataset['clip_path'].iloc[i]
            class_name = dataset['label'].iloc[i]

            cap = cv2.VideoCapture(video_path)

            if not cap.isOpened():
                print(f"Error: Could not open video file {video_path}")
                continue  # Skip to the next video file if there is an error

            print(f"Processing video: {video_path} ({class_name})")
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret or frame is None:
                    print(f"End of video or error reading frame {frame_count} from {video_path}")
                    break

                frame_count += 1
                try:
                    # Resize frame
                    frame = cv2.resize(frame, (640, 480))

                    # Convert to uint8 if necessary
                    if frame.dtype != np.uint8:
                        frame = (frame * 255).astype(np.uint8)

                    # Apply Median Blur
                    frame = cv2.medianBlur(frame, 5)
                
                    # Convert to RGB for MediaPipe
                    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    image.flags.writeable = False

                    # Process the frame with MediaPipe Holistic
                    results = holistic.process(image)
                    image.flags.writeable = True
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                    # Draw pose landmarks if present
                    if results.pose_landmarks:
                        mp_drawing.draw_landmarks(
                            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                        )

                        # Extract pose landmarks
                        try:
                            pose = results.pose_landmarks.landmark
                            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

                            x = pd.DataFrame([pose_row])
                            body_language_class = model.predict(x)[0]
                            body_language_prob = model.predict_proba(x)[0]
                            print(f"Class: {body_language_class}, Probability: {body_language_prob}")

                            # Annotate the frame
                            cv2.putText(image, f'Class: {body_language_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                            cv2.putText(image, f'Probability: {body_language_prob[np.argmax(body_language_prob)]:.2f}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                        except Exception as e:
                            print(f"Error extracting pose landmarks: {e}")

                    # Display the video frame
                    cv2.imshow('Processed Video Feed', image)

                    # Break the loop on 'q' key press
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break
                except cv2.error as e:
                    print(f"OpenCV error at frame {frame_count}: {e}")
                    break

            cap.release()
            print(f"Finished processing video: {video_path}")

    cv2.destroyAllWindows()

# Call the function with a dataset and the desired amount
def gradio_interface():
    return test_videos(dataset_test, 20)

iface = gr.Interface(
    fn=gradio_interface,
    inputs=None,
    outputs=gr.Video(label="Labelled Video"),
    live=True,
    description="Click the button to process the video."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Processing video: ./test/Swing/v_Swing_g21_c02.avi (Swing)


c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: U

Class: Swing, Probability: [0.21699567 0.10416667 0.50064719 0.17819048]
Class: Swing, Probability: [0.14472294 0.16962121 0.52928355 0.15637229]
Class: Swing, Probability: [0.09472294 0.18295455 0.56595022 0.15637229]


c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: U

...

Class: Swing, Probability: [0.06063492 0.09184127 0.72047619 0.12704762]
Class: Swing, Probability: [0.00837607 0.02272727 0.72955983 0.23933683]
Finished processing video: ./test/Swing/v_Swing_g21_c06.avi
Processing video: ./test/Swing/v_Swing_g21_c04.avi (Swing)
Class: TableTennisShot, Probability: [0.01863636 0.05484848 0.43989177 0.48662338]
Finished processing video: ./test/Swing/v_Swing_g21_c04.avi
Processing video: ./test/Swing/v_Swing_g20_c06.avi (Swing)
Finished processing video: ./test/Swing/v_Swing_g20_c06.avi
Processing video: ./test/Swing/v_Swing_g14_c02.avi (Swing)
Finished processing video: ./test/Swing/v_Swing_g14_c02.avi
Processing video: ./test/Swing/v_Swing_g04_c05.avi (Swing)
Finished processing video: ./test/Swing/v_Swing_g04_c05.avi
Processing video: ./test/Swing/v_Swing_g12_c03.avi (Swing)


c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Class: Swing, Probability: [0.00181818 0.16874359 0.6190692  0.21036903]
Finished processing video: ./test/Swing/v_Swing_g12_c03.avi
Processing video: ./test/Swing/v_Swing_g19_c01.avi (Swing)
Finished processing video: ./test/Swing/v_Swing_g19_c01.avi
Processing video: ./test/Swing/v_Swing_g20_c05.avi (Swing)
Finished processing video: ./test/Swing/v_Swing_g20_c05.avi
Processing video: ./test/Swing/v_Swing_g04_c04.avi (Swing)
Finished processing video: ./test/Swing/v_Swing_g04_c04.avi
Processing video: ./test/Swing/v_Swing_g20_c01.avi (Swing)
Finished processing video: ./test/Swing/v_Swing_g20_c01.avi
Processing video: ./test/Swing/v_Swing_g19_c03.avi (Swing)
Finished processing video: ./test/Swing/v_Swing_g19_c03.avi
Processing video: ./test/Swing/v_Swing_g23_c01.avi (Swing)


c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Class: Swing, Probability: [0.04       0.07405983 0.60760684 0.27833333]
Finished processing video: ./test/Swing/v_Swing_g23_c01.avi
Processing video: ./test/Swing/v_Swing_g11_c04.avi (Swing)
Finished processing video: ./test/Swing/v_Swing_g11_c04.avi
Processing video: ./test/Swing/v_Swing_g01_c02.avi (Swing)
Finished processing video: ./test/Swing/v_Swing_g01_c02.avi
Processing video: ./test/Swing/v_Swing_g04_c03.avi (Swing)
Class: Swing, Probability: [0.07204545 0.19084005 0.63822561 0.09888889]
Finished processing video: ./test/Swing/v_Swing_g04_c03.avi
Processing video: ./test/Swing/v_Swing_g12_c01.avi (Swing)
Finished processing video: ./test/Swing/v_Swing_g12_c01.avi
Processing video: ./test/SkyDiving/v_SkyDiving_g16_c03.avi (SkyDiving)
Finished processing video: ./test/SkyDiving/v_SkyDiving_g16_c03.avi
Processing video: ./test/SkyDiving/v_SkyDiving_g25_c04.avi (SkyDiving)
Finished processing video: ./test/SkyDiving/v_SkyDiving_g25_c04.avi
Processing video: ./test/SkyDiving/v_Sky

In [14]:
import gradio as gr
import cv2
import numpy as np
import mediapipe as mp
import tempfile
import os

# Mediapipe setup
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# Function to convert avi to mp4
def convert_avi_to_mp4(avi_file):
    # Create a temporary file for the output mp4
    temp_mp4_path = tempfile.mktemp(suffix=".mp4")
    cap = cv2.VideoCapture(avi_file.name)
    
    # Check if the video file is opened correctly
    if not cap.isOpened():
        return None
    
    # Get frame width, height, and FPS for video saving
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(temp_mp4_path, fourcc, fps, (width, height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
    
    cap.release()
    out.release()
    
    return temp_mp4_path

# Function to process video and recognize activity
def recognize_pose(video_file):
    # Convert avi to mp4 (Gradio supports mp4)
    temp_mp4_path = convert_avi_to_mp4(video_file)
    if temp_mp4_path is None:
        return "Error: Unable to process video."

    # Now process the mp4 video
    with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
        cap = cv2.VideoCapture(temp_mp4_path)
        output_frames = []

        # Check if video is loaded correctly
        if not cap.isOpened():
            return "Error: Unable to read the video."

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Resize and preprocess frame
            frame = cv2.resize(frame, (640, 480))
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Process frame with Mediapipe
            results = holistic.process(image)

            # If pose landmarks are detected, draw them
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                )
            else:
                print("No pose landmarks detected in this frame.")

            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Add processed frame to output frames list
            output_frames.append(image)

        cap.release()

    # Ensure there are frames in the output
    if not output_frames:
        return "Error: No frames processed."

    # Save the processed frames to the temporary video file
    temp_output_path = tempfile.mktemp(suffix=".mp4")
    height, width, layers = output_frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using the MP4 codec
    out = cv2.VideoWriter(temp_output_path, fourcc, 20.0, (width, height))

    for frame in output_frames:
        out.write(frame)
    out.release()

    # Clean up the temporary mp4 conversion file
    os.remove(temp_mp4_path)

    # Return the path to the processed video
    return temp_output_path

# Gradio interface function
def gradio_interface(video):
    result_video = recognize_pose(video)
    return result_video

# Gradio setup
interface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Video(label="Input Video"),
    outputs=gr.Video(label="Processed Video")
)

# Launch the Gradio app
interface.launch()


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 2047, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\devin\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1594, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\devin\AppData\Local\P